## Lab 6

### Imports

In [ ]:
!pip install transformers datasets simpletransformers==0.65.1
import pandas as pd
import numpy as np
import sklearn
from sklearn.metrics import classification_report
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import matplotlib.pyplot as plt
import seaborn as sn
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.6/312.6 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 5.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 699.5 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 875.8

### Code

In [ ]:
from sklearn.datasets import fetch_20newsgroups

categories = ['alt.atheism', 'comp.graphics', 'sci.med', 'sci.space']

newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'), categories=categories, random_state=42)
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'), categories=categories, random_state=42)

In [ ]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

In [ ]:
train_df = pd.DataFrame({
    'text': newsgroups_train.data,
    'label': newsgroups_train.target
})

test_df = pd.DataFrame({
    'text': newsgroups_test.data,
    'label': newsgroups_test.target
})


In [ ]:
model_args = ClassificationArgs()
model_args.num_train_epochs = 3
model_args.learning_rate = 4e-5
model_args.overwrite_output_dir = True
model_args.train_batch_size = 16
model_args.eval_batch_size = 8

model = ClassificationModel(
    "roberta",
    "roberta-base",
    num_labels=len(categories),
    args=model_args,
    use_cuda=True,
)
model.train_model(train_df)

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:608: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/4 [00:00<?, ?it/s]

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Running Epoch 1 of 3:   0%|          | 0/141 [00:00<?, ?it/s]

Running Epoch 2 of 3:   0%|          | 0/141 [00:00<?, ?it/s]

Running Epoch 3 of 3:   0%|          | 0/141 [00:00<?, ?it/s]

(423, 0.3555804880173001)

In [ ]:
result, model_outputs, wrong_predictions = model.eval_model(test_df, verbose=True)

/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:1452: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/2 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/188 [00:00<?, ?it/s]

In [ ]:
predictions, raw_outputs = model.predict(test_df['text'].tolist())
print(classification_report(test_df['label'], predictions, target_names=categories))

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/188 [00:00<?, ?it/s]

               precision    recall  f1-score   support

  alt.atheism       0.82      0.81      0.82       319
comp.graphics       0.91      0.92      0.92       389
      sci.med       0.89      0.89      0.89       396
    sci.space       0.84      0.84      0.84       394

     accuracy                           0.87      1498
    macro avg       0.86      0.86      0.86      1498
 weighted avg       0.87      0.87      0.87      1498



### Analysis


### Summary of Model Performances

#### BERT (Lab6.4):
- **Precision**: Ranges from 0.83 to 0.90 across categories.
- **Recall**: Ranges from 0.79 to 0.89.
- **F1-Score**: Ranges from 0.81 to 0.90.
- **Overall Accuracy**: 0.85.

#### RoBERTa (Lab6_g47):
- **Precision**: Ranges from 0.78 to 0.92.
- **Recall**: Ranges from 0.81 to 0.89.
- **F1-Score**: Ranges from 0.79 to 0.91.
- **Overall Accuracy**: 0.85.

#### SVM (ConventionalSVM):
- **Precision**: Ranges from 0.74 to 0.88.
- **Recall**: Ranges from 0.76 to 0.87.
- **F1-Score**: Ranges from 0.80 to 0.87.
- **Overall Accuracy**: 0.83.

### Analysis

Accuracy and F1-Score:

Transformer models (BERT and RoBERTa) show superior overall accuracy and F1-scores compared to the SVM model, indicating a better balance between precision and recall.

 Precision and Recall Trade-offs:

  - BERT exhibits slightly higher precision for 'alt.atheism' and 'sci.med', suggesting efficient identification of relevant instances.
  - RoBERTa demonstrates higher recall in certain categories, indicating its effectiveness in retrieving more relevant instances, likely due to its advanced contextual understanding.
  - SVM, while competitive, generally shows lower metrics, particularly in 'sci.space', possibly due to limitations in handling ambiguous content with bag-of-words features.

 Model Suitability:

  - Transformer Models (BERT and RoBERTa): Best suited for tasks requiring deep textual understanding and contextualization, benefiting from their pre-training on extensive text corpora.
  - SVM: Suitable for scenarios with limited computational resources or where model interpretability is crucial. It remains a robust baseline for simpler text classification problems.


#### Conclusion

Transformer-based models, BERT and RoBERTa, outperform the conventional SVM approach in the text classification task, highlighting their superior language understanding capabilities. The choice of model should, however, consider the specific requirements of the task, such as computational constraints, interpretability, and the complexity of the text data.
